In [20]:
import numpy as np
import os
import spectral
import imageio
import sklearn
import sklearn.model_selection
from sklearn import preprocessing
from scipy.stats import entropy


In [ ]:
def softmax_new(x):
    ret = np.zeros(x.shape)
    filt = x == x
    x_new = np.nan_to_num(x[np.where(filt)])
    x_row_max = np.nanmax(x_new, axis=-1)
    x_row_max = x_row_max.reshape(list(x_new.shape)[:-1] + [1])
    x_new = x_new - x_row_max
    x_exp = np.exp(x_new)
    x_exp_row_sum = x_exp.sum(axis=-1).reshape(list(x.shape)[:-1] + [1])
    softmax = x_exp / x_exp_row_sum
    ret[np.where(filt)] = softmax
    return ret
def sample_gt3_new(img, gt, train_gt, test_gt, SAMPLE_PERCENTAGE, IGNORED_LABELS, ALL_LABELS):
    X = img
    Y = gt
    labels = np.unique(Y)
    labels = np.array([val for val in labels if not val in IGNORED_LABELS])
    print(labels)
    row, col, n_band = X.shape
    num_class = len(ALL_LABELS) # TODO: Fix to the correct amount
    max_class = np.max(ALL_LABELS)
    # num_class = len(np.unique(labels))
    first = True
    skipped_labels = []
    for i, val in enumerate(ALL_LABELS): # range(1, num_class + 1):
        if val in labels:
            index = np.where(train_gt == val)
            index2 = np.where(test_gt == val)
            if first:
                array1_train = index[0]
                array2_train = index[1]
                array1_test = index2[0]
                array2_test = index2[1]
                first = False
            else:
                array1_train = np.concatenate((array1_train, index[0]))
                array2_train = np.concatenate((array2_train, index[1]))
                array1_test = np.concatenate((array1_test, index2[0]))
                array2_test = np.concatenate((array2_test, index2[1]))
        else:
            skipped_labels.append(val)
    y_train = Y[array1_train, array2_train]
    trueEDtimesSID = []
    trueEDtimesSID2 = []
    sumtrueES = []
    sumfalseES = []
    pseudo_labels3 = np.zeros([row, col, num_class])
    for i in range(0, len(array1_test)):
        if i % 1000 == 0:
            print("i:%d" % (i))
        #if i%200!=0:
        #    continue
        xtest = array1_test[i]
        ytest = array2_test[i]
        labeltest = Y[xtest, ytest]
        specvectortest = X[xtest, ytest]
        # i
        EDs = np.zeros(num_class)
        SIDs = np.zeros(num_class)
        EDtimesSIDs = np.zeros(num_class)
        EDtimesSIDs2 = np.zeros(num_class)
        minED = 10000000000
        for j, val in enumerate(ALL_LABELS): #range(1, num_class + 1):  # 类别循环
            if val in labels:
                index2 = np.where(y_train == val)  ## 当前类别序号
                index2 = index2[0]
                EDsclass = []
                SIDclass = []
                EDtimesSIDclass = []
                for nn in range(0, len(index2)):  # 类别内训练集循环 nn
                    # print(index2[nn])##当前训练样本序号
                    ind = index2[nn]
                    xtrain = array1_train[ind]
                    ytrain = array2_train[ind]
                    specvectortrain = X[xtrain, ytrain]
                    ED = np.sqrt(np.square(xtest - xtrain) + np.square(ytest - ytrain))
                    SID1 = entropy(specvectortest, specvectortrain)
                    SID2 = entropy(specvectortrain, specvectortest)
                    SID = SID1 + SID2
                    EDtimesSID = np.sqrt(ED * SID)
                    ED = ED + SID
                    EDsclass.append(ED)
                    SIDclass.append(SID)
                    EDtimesSIDclass.append(EDtimesSID)

                    if ED < minED:
                        minED = ED
            # =================================
                inde = np.argsort(EDsclass)

                jiaquan = 0
                for nn in range(0, len(index2)):
                    jiaquandis = EDsclass[inde[nn]] * (float(num_class) ** (-nn))  # 类别内训练集循环 nn
                    jiaquan = jiaquan + jiaquandis

                EDs[j] = jiaquan
                SIDs[j] = np.min(SIDclass)
                EDtimesSIDs[j] = np.min(EDtimesSIDclass)
                jiaquan2 = 0
                inde2 = np.argsort(EDtimesSIDclass)
                for nn in range(0, len(index2)):
                    jiaquandis = EDtimesSIDclass[inde2[nn]] * (float(num_class) ** (-nn))  # 类别内训练集循环 nn
                    jiaquan2 = jiaquan2 + jiaquandis
                EDtimesSIDs2[j] = jiaquan2
            else:
                EDs[j] = np.nan
                SIDs[j] = np.nan
                EDtimesSIDs[j] = np.nan
                EDtimesSIDs2[j] = np.nan
        if np.nanmin(EDtimesSIDs) > 0.085:
            continue
        else:
            minn = np.nanmin(EDs)
            softm3 = softmax_new(-EDtimesSIDs2 * max_class*100)
            minn
        labeEDtimesSIDs = np.nanargmin(EDtimesSIDs)
        labeEDtimesSIDs2 = np.nanargmin(EDtimesSIDs2)
        train_gt[xtest, ytest] = labeEDtimesSIDs2
        pseudo_labels3[xtest, ytest][ALL_LABELS] = softm3


        if labeEDtimesSIDs == labeltest:
            trueEDtimesSID.append(1)
            sumtrueES.append(np.nanmin(EDtimesSIDs))
        else:
            trueEDtimesSID.append(0)
            sumfalseES.append(np.nanmin(EDtimesSIDs))
            print("falseEDtimesSID:", np.nanmin(EDtimesSIDs))

        if labeEDtimesSIDs2 == labeltest:
            trueEDtimesSID2.append(1)
        else:
            trueEDtimesSID2.append(0)
    accuEDtimesSID2 = np.sum(trueEDtimesSID2) / len(trueEDtimesSID2)


In [ ]:
def sample_gt(gt, train_size, mode='random'):
    """Extract a fixed percentage of samples from an array of labels.

    Args:
        gt: a 2D array of int labels
        percentage: [0, 1] float
    Returns:
        train_gt, test_gt: 2D arrays of int labels

    """

    indices = np.nonzero(gt)
    X = list(zip(*indices)) # x,y features
    y = gt[indices].ravel() # classes
    train_gt = np.zeros_like(gt)
    test_gt = np.zeros_like(gt)
    if train_size > 1:
       train_size = int(train_size)
    
    if mode == 'random':
       train_indices, test_indices = sklearn.model_selection.train_test_split(X, train_size=train_size, stratify=y)
       train_indices = [list(t) for t in zip(*train_indices)]
       test_indices = [list(t) for t in zip(*test_indices)]
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]
    elif mode == 'fixed':
       # print("Sampling {} with train size = {}".format(mode, train_size))
       train_indices, test_indices = [], []
        for c in np.unique(gt):
            if c == 0:
                continue
           indices = np.nonzero(gt == c)
           X = list(zip(*indices)) # x,y features
           ##================
           classsize = len(X)
           train_size2=train_size
           if classsize<=train_size:
               train_size2=classsize-1
           ##====================
           train, test = sklearn.model_selection.train_test_split(X, train_size=train_size2)
           train_indices += train
           test_indices += test
       train_indices = [list(t) for t in zip(*train_indices)]
       test_indices = [list(t) for t in zip(*test_indices)]
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]

    elif mode == 'disjoint':
        train_gt = np.copy(gt)
        test_gt = np.copy(gt)
        for c in np.unique(gt):
            mask = gt == c
            for x in range(gt.shape[0]):
                first_half_count = np.count_nonzero(mask[:x, :])
                second_half_count = np.count_nonzero(mask[x:, :])
                try:
                    ratio = first_half_count / second_half_count
                    if ratio > 0.9 * train_size and ratio < 1.1 * train_size:
                        break
                except ZeroDivisionError:
                    continue
            mask[:x, :] = 0
            train_gt[mask] = 0

        test_gt[train_gt > 0] = 0
    else:
        raise ValueError("{} sampling is not implemented yet.".format(mode))
    return train_gt, test_gt


In [4]:
def open_file(dataset): #TODO: implement opening dataset containing multiple files for tif files
    _, ext = os.path.splitext(dataset)
    ext = ext.lower()
    if ext == '.mat':
        # Load Matlab array
        return io.loadmat(dataset)
    elif ext == '.tif' or ext == '.tiff':
        # Load TIFF file
        return imageio.imread(dataset)
    elif ext == '.hdr':
        img = spectral.open_image(dataset)
        return img.load()
    else:
        raise ValueError("Unknown file format: {}".format(ext))

In [5]:
DATASETS_CONFIG = {
        'PaviaC': {
            'urls': ['http://www.ehu.eus/ccwintco/uploads/e/e3/Pavia.mat', 
                     'http://www.ehu.eus/ccwintco/uploads/5/53/Pavia_gt.mat'],
            'img': 'Pavia.mat',
            'gt': 'Pavia_gt.mat'
            },
        'PaviaU': {
            'urls': ['http://www.ehu.eus/ccwintco/uploads/e/ee/PaviaU.mat',
                     'http://www.ehu.eus/ccwintco/uploads/5/50/PaviaU_gt.mat'],
            'img': 'PaviaU.mat',
            'gt': 'PaviaU_gt.mat'
            },
        'KSC': {
            'urls': ['http://www.ehu.es/ccwintco/uploads/2/26/KSC.mat',
                     'http://www.ehu.es/ccwintco/uploads/a/a6/KSC_gt.mat'],
            'img': 'KSC.mat',
            'gt': 'KSC_gt.mat'
            },
        'IndianPines': {
            'urls': ['http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat',
                     'http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat'],
            'img': 'Indian_pines_corrected.mat',
            'gt': 'Indian_pines_gt.mat'
            },
        'Botswana': {
            'urls': ['http://www.ehu.es/ccwintco/uploads/7/72/Botswana.mat',
                     'http://www.ehu.es/ccwintco/uploads/5/58/Botswana_gt.mat'],
            'img': 'Botswana.mat',
            'gt': 'Botswana_gt.mat',
            },
        'Houston': {
            'urls': ['http://www.ehu.es/ccwintco/uploads/7/72/Botswana.mat',
                 'http://www.ehu.es/ccwintco/uploads/5/58/Botswana_gt.mat'],
            'img': 'Houston.mat',
            'gt': 'Houston_gt.mat',
            },
        'hyrank': {
            'urls': ['', ''],
            'img': 'Training Set/Anafi.tif',
            'gt': 'Training Set/Anafi_GT.tif'
        }
    }


In [6]:
def get_dataset(dataset_name, target_folder="./", datasets=DATASETS_CONFIG):
    # TODO: Add Hyrank
    """ Gets the dataset specified by name and return the related components.
    Args:
        dataset_name: string with the name of the dataset
        target_folder (optional): folder to store the datasets, defaults to ./
        datasets (optional): dataset configuration dictionary, defaults to prebuilt one
    Returns:
        img: 3D hyperspectral image (WxHxB)
        gt: 2D int array of labels
        label_values: list of class names
        ignored_labels: list of int classes to ignore
        rgb_bands: int tuple that correspond to red, green and blue bands
    """
    palette = None
    
    if dataset_name not in list(datasets.keys()):
        raise ValueError("{} dataset is unknown.".format(dataset_name))

    dataset = datasets[dataset_name]

    folder = target_folder + datasets[dataset_name].get('folder', dataset_name + '/')
    if dataset.get('download', True):
        # Download the dataset if is not present
        if not os.path.isdir(folder):
            os.mkdir(folder)
        for url in datasets[dataset_name]['urls']:
            # download the files
            filename = url.split('/')[-1]
            if not os.path.exists(folder + filename):
                with TqdmUpTo(unit='B', unit_scale=True, miniters=1,
                          desc="Downloading {}".format(filename)) as t:
                    urlretrieve(url, filename=folder + filename,
                                     reporthook=t.update_to)
    elif not os.path.isdir(folder):
       print("WARNING: {} is not downloadable.".format(dataset_name))

    if dataset_name == 'PaviaC':
        # Load the image
        img = open_file(folder + 'Pavia.mat')['pavia']

        rgb_bands = (55, 41, 12)

        gt = open_file(folder + 'Pavia_gt.mat')['pavia_gt']

        label_values = ["Undefined", "Water", "Trees", "Asphalt",
                        "Self-Blocking Bricks", "Bitumen", "Tiles", "Shadows",
                        "Meadows", "Bare Soil"]
        all_labels = np.arange(len(label_values))

        ignored_labels = [0]
        img = [img]
        gt = [gt]

    elif dataset_name == 'PaviaU':
        # Load the image
        img = open_file(folder + 'PaviaU.mat')['paviaU']

        rgb_bands = (55, 41, 12)

        gt = open_file(folder + 'PaviaU_gt.mat')['paviaU_gt']

        label_values = ['Undefined', 'Asphalt', 'Meadows', 'Gravel', 'Trees',
                        'Painted metal sheets', 'Bare Soil', 'Bitumen',
                        'Self-Blocking Bricks', 'Shadows']
        all_labels = np.arange(len(label_values))

        ignored_labels = [0]
        img = [img]
        gt = [gt]

    elif dataset_name == 'IndianPines':
        # Load the image
        img = open_file(folder + 'Indian_pines_corrected.mat')
        img = img['indian_pines_corrected']

        rgb_bands = (43, 21, 11)  # AVIRIS sensor

        gt = open_file(folder + 'Indian_pines_gt.mat')['indian_pines_gt']
        label_values = ["Undefined", "Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]
        all_labels = np.arange(len(label_values))

        ignored_labels = [0]
        img = [img]
        gt = [gt]
    elif dataset_name == 'Houston':
        # Load the image
        img = open_file(folder + 'Houston.mat')
        img = img['img']

        rgb_bands = (43, 21, 11)  # AVIRIS sensor

        gt = open_file(folder + 'Houston_gt.mat')['Houston_gt']
        label_values = ["Undefined", "1", "Corn-2", "Corn-3",
                        "4", "Grass-5", "6-trees",
                        "7-pasture-mowed", "Hay-8", "9",
                        "Soybean-10", "11-mintill", "12-clean",
                        "13", "14", "15-Grass-Trees-Drives"]
        all_labels = np.arange(len(label_values))

        ignored_labels = [0]
        img = [img]
        gt = [gt]
    elif dataset_name == 'Botswana':
        # Load the image
        img = open_file(folder + 'Botswana.mat')['Botswana']

        rgb_bands = (75, 33, 15)

        gt = open_file(folder + 'Botswana_gt.mat')['Botswana_gt']
        label_values = ["Undefined", "Water", "Hippo grass",
                        "Floodplain grasses 1", "Floodplain grasses 2",
                        "Reeds", "Riparian", "Firescar", "Island interior",
                        "Acacia woodlands", "Acacia shrublands",
                        "Acacia grasslands", "Short mopane", "Mixed mopane",
                        "Exposed soils"]

        all_labels = np.arange(len(label_values))
        ignored_labels = [0]
        img = [img]
        gt = [gt]
    elif dataset_name == 'KSC':
        # Load the image
        img = open_file(folder + 'KSC.mat')['KSC']

        rgb_bands = (43, 21, 11)  # AVIRIS sensor

        gt = open_file(folder + 'KSC_gt.mat')['KSC_gt']
        label_values = ["Undefined", "Scrub", "Willow swamp",
                        "Cabbage palm hammock", "Cabbage palm/oak hammock",
                        "Slash pine", "Oak/broadleaf hammock",
                        "Hardwood swamp", "Graminoid marsh", "Spartina marsh",
                        "Cattail marsh", "Salt marsh", "Mud flats", "Wate"]

        all_labels = np.arange(len(label_values))
        ignored_labels = [0]
        img = [img]
        gt = [gt]
    elif dataset_name == 'hyrank':
        files = ['Training Set/Anafi.tif',
                 'Training Set/Atokos.tif',
                 'Training Set/Donousa.tif',
                 'Validation Set/Kasos.tif',
                 'Validation Set/Tilos.tif',
                ]
        
        files_gt = ['Training Set/Anafi_GT.tif',
                    'Training Set/Atokos_GT.tif',
                    'Training Set/Donousa_GT.tif',
                    'Validation Set/Kasos_GT.tif',
                    'Validation Set/Tilos_GT.tif',
                    ]
        img = [open_file(folder + i) for i in files]

        rgb_bands = [30,20,2]

        gt = [open_file(folder + i) for i in files_gt]

        label_values = ['Undefined',
                        'High intensity developed', 
                        'Med-low intensity developed',
                        'Deciduous, Evergreen, mixed forest',
                        'shrubland',
                        'Grassland-Pasture',
                        'Bareland',
                        'water',
                        'corn',
                        'cotton',
                        'cereals',
                        'almonds',
                        'grass fodders',
                        'vinewards-grapes',
                        'walnuts',
                        'pistachios',
                        'citrus',
                        'fallow']

        all_labels = np.arange(len(label_values))


        ignored_labels = [0]
    else:
        # Custom dataset
        img, gt, rgb_bands, ignored_labels, label_values, palette = CUSTOM_DATASETS_CONFIG[dataset_name]['loader'](folder)
        img = [img]
        gt = [gt]
    # Filter NaN out
    for i, im in enumerate(img):
        nan_mask = np.isnan(im.sum(axis=-1))
        if np.count_nonzero(nan_mask) > 0:
            print("Warning: NaN have been found in the data. It is preferable to remove them beforehand. Learning on NaN data is disabled.")
            img[i][nan_mask] = 0
            gt[i][nan_mask] = 0
            ignored_labels.append(0)

        # Normalization
        im = np.asarray(im, dtype='float32')
        #img = (img - np.min(img)) / (np.max(img) - np.min(img))
        data = im.reshape(np.prod(im.shape[:2]), np.prod(im.shape[2:]))
        #data = preprocessing.scale(data)
        data  = preprocessing.minmax_scale(data)
        img[i] = data.reshape(im.shape)
    ignored_labels = list(set(ignored_labels))
    return img, gt, label_values, ignored_labels, all_labels, rgb_bands, palette


def get_originate_dataset(dataset_name, target_folder="./", datasets=DATASETS_CONFIG):


    if dataset_name not in list(datasets.keys()):
        raise ValueError("{} dataset is unknown.".format(dataset_name))

    dataset = datasets[dataset_name]

    folder = target_folder + datasets[dataset_name].get('folder', dataset_name + '/')


    if dataset_name == 'PaviaC':
        img = open_file(folder + 'Pavia.mat')['pavia']
        gt = open_file(folder + 'Pavia_gt.mat')['pavia_gt']
        img = [img]
        gt = [gt]
    elif dataset_name == 'PaviaU':
        img = open_file(folder + 'PaviaU.mat')['paviaU']
        gt = open_file(folder + 'PaviaU_gt.mat')['paviaU_gt']
        img = [img]
        gt = [gt]
    elif dataset_name == 'IndianPines':
        img = open_file(folder + 'Indian_pines_corrected.mat')
        img = img['indian_pines_corrected']
        rgb_bands = (43, 21, 11)  # AVIRIS sensor
        gt = open_file(folder + 'Indian_pines_gt.mat')['indian_pines_gt']
        img = [img]
        gt = [gt]
    elif dataset_name == 'Botswana':
        img = open_file(folder + 'Botswana.mat')['Botswana']
        gt = open_file(folder + 'Botswana_gt.mat')['Botswana_gt']
        img = [img]
        gt = [gt]
    elif dataset_name == 'KSC':
        img = open_file(folder + 'KSC.mat')['KSC']
        gt = open_file(folder + 'KSC_gt.mat')['KSC_gt']
        img = [img]
        gt = [gt]
    elif dataset_name == 'Houston':
        img = open_file(folder + 'Houston.mat')
        img = img['img']
        gt = open_file(folder + 'Houston_gt.mat')['Houston_gt']
        img = [img]
        gt = [gt]
    elif dataset_name == 'hyrank':
        files = ['Training Set/Anafi.tif',
                 'Training Set/Atokos.tif',
                 'Training Set/Donousa.tif',
                 'Validation Set/Kasos.tif',
                 'Validation Set/Tilos.tif',
                ]
        
        files_gt = ['Training Set/Anafi_GT.tif',
                    'Training Set/Atokos_GT.tif',
                    'Training Set/Donousa_GT.tif',
                    'Validation Set/Kasos_GT.tif',
                    'Validation Set/Tilos_GT.tif',
                    ]
        img = [open_file(folder + i) for i in files]
        gt = [open_file(folder + i) for i in files_gt]
    return img, gt


In [17]:
def load_datasets(DATASET, datasets_root, SAMPLE_PERCENTAGE):
    img, gt, LABEL_VALUES, IGNORED_LABELS, ALL_LABELS, _, _ = get_dataset(DATASET, datasets_root)
    X, Y = get_originate_dataset(DATASET, datasets_root)
    img = concat_with_padding(img, [1])
    img = img[:, :, list(range(0, 102, 3))] # Why is this here? This takes the every third channel from every dataset.
                                            # The end point being fixed at 102 might be a problem. 
                                            # On the other hand no similar thing is done to X, which is supposedly
                                            # the same image. 

    N_CLASSES = len(LABEL_VALUES)
    INPUT_SIZE = np.shape(img)[-1]
    # train_gt, test_gt = sample_gt(gt, SAMPLE_PERCENTAGE, mode='fixed')
    train_test_gt = [sample_gt(i, SAMPLE_PERCENTAGE, mode='fixed') for i in gt]
    #train_gt = sample_gt2(X, Y, train_gt, test_gt, SAMPLE_PERCENTAGE)
    # # # breakpoint()
    pseudo_labelpath= '../' + str(DATASET) + f'/pseudo_labels/pseudo_labels3/pseudo_labels3_{SAMPLE_PERCENTAGE}.npy'
    pseudo_labels3 = []
    if not os.path.exists(pseudo_labelpath):
        newdir = str(DATASET) + '/pseudo_labels/pseudo_labels3/'
        if not os.path.exists(newdir):
            os.makedirs(newdir)
        for x, y, tr_te_gt in zip(X, Y, train_test_gt):
            pseudo_labels3.append(sample_gt3_new(x, y, tr_te_gt[0], tr_te_gt[1], 
                                                 SAMPLE_PERCENTAGE, IGNORED_LABELS, ALL_LABELS))
        pseudo_labels3 = concat_with_padding(pseudo_labels3, [1])
        np.save(pseudo_labelpath, pseudo_labels3)
    else:
        pseudo_labels3=np.load(pseudo_labelpath)
    train_gt = concat_with_padding([i[0] for i in train_test_gt], [1])
    test_gt = concat_with_padding([i[1] for i in train_test_gt], [1])
    gt = concat_with_padding(gt, [1]) 
    X = concat_with_padding(X, [1])
    Y = concat_with_padding(Y, [1])
    return img, gt, LABEL_VALUES, IGNORED_LABELS, ALL_LABELS, X, Y, N_CLASSES,\
           INPUT_SIZE, train_gt, test_gt, pseudo_labels3


In [18]:
def single_pad(pad_value):
        if pad_value%2 == 0:
            return [int(pad_value/2), int(pad_value/2)]
        else:
            return [int(pad_value/2), int(pad_value/2) + 1]
def calc_pad(mx, shape, axes):
    pad_values = mx - shape
    pads = [[0,0], [0,0], [0,0]]
    for axis in axes:
        #print(i)
        #print(i in axes)
        pads[axis] = (single_pad(pad_values[axis]))
    # print(pads)
    return pads
    
def concat_with_padding(list_of_arrays, padding_axes):
    shape1s = [i.shape for i in list_of_arrays]
    max_shape = np.max(shape1s, axis=0)
    #max_shape = np.array(img[-1].shape)
    pads = [calc_pad(max_shape, i, padding_axes) for i in shape1s]
    img2 = [np.pad(list_of_arrays[i], pad) for i, pad in enumerate(pads)]
    res2 = np.concatenate(img2)
    return res2

In [21]:
img, gt, LABEL_VALUES, IGNORED_LABELS, ALL_LABELS,\
X, Y, N_CLASSES, INPUT_SIZE, train_gt, test_gt, pseudo_labels3 = load_datasets('hyrank', '/mnt/data/leevi/', 5)

/tmp/ipykernel_1024889/2054537087.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(dataset)
/tmp/ipykernel_1024889/2054537087.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(dataset)
/tmp/ipykernel_1024889/1608058974.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  train_gt[train_indices] = gt[train_indice

[ 1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17]
i:0
i:1000
i:2000
i:3000
i:4000
i:5000
i:6000
i:7000
i:8000
i:9000
i:10000
i:11000
i:12000
i:13000
i:14000
i:15000
i:16000
i:17000
i:18000
i:19000
i:20000
i:21000
i:22000
i:23000
i:24000
i:25000
i:26000
i:27000
i:28000
i:29000
i:30000
i:31000
i:32000
i:33000
i:34000
i:35000
i:36000
i:37000
i:38000
i:39000
i:40000
i:41000
i:42000
i:43000
i:44000
i:45000
i:46000
i:47000
i:48000
i:49000
i:50000
i:51000
i:52000
i:53000
i:54000
i:55000
i:56000
i:57000
i:58000
i:59000
i:60000
i:61000
i:62000
i:63000
i:64000
i:65000
i:66000
i:67000
i:68000
i:69000
i:70000
i:71000
i:72000
i:73000
i:74000
i:75000
i:76000
i:77000
i:78000
i:79000
i:80000
i:81000
i:82000
i:83000
i:84000
i:85000
i:86000
i:87000
i:88000
i:89000
i:90000
i:91000
i:92000
i:93000
i:94000
i:95000
i:96000
i:97000
i:98000
i:99000
i:100000
i:101000
i:102000
i:103000
i:104000
i:105000
i:106000
i:107000
i:108000
i:109000
i:110000
i:111000
i:112000
i:113000
i:114000
i:115000
i:116000
i:117000
i

KeyboardInterrupt: 

In [8]:
folder = '/mnt/data/leevi/hyrank/'
files = ['Training Set/Anafi.tif',
                 'Training Set/Atokos.tif',
                 'Training Set/Donousa.tif',
                 'Validation Set/Kasos.tif',
                 'Validation Set/Tilos.tif',
                ]
        
files_gt = ['Training Set/Anafi_GT.tif',
            'Training Set/Atokos_GT.tif',
            'Training Set/Donousa_GT.tif',
            'Validation Set/Kasos_GT.tif',
            'Validation Set/Tilos_GT.tif',
            ]
img = [open_file(folder + i) for i in files]
gt = [open_file(folder + i) for i in files_gt]

/tmp/ipykernel_1024889/2054537087.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(dataset)


In [9]:
for i in range(5):
    print(img[i].shape)
    print(gt[i].shape)
    print('-------')

(16640, 710, 147)
(16640, 710)
-------
(12000, 700, 147)
(12000, 700)
-------
(10800, 660, 147)
(10800, 660)
-------
(11620, 680, 147)
(11620, 680)
-------
(14700, 740, 147)
(14700, 740)
-------


In [ ]:
import time
start = time.time()
max_shape1 = 0
for i, im in enumerate(img):
    max_updated = False
    max_equal = False
    if max_shape1 == im.shape[1]:
        max_equal = True
    if max_shape1 < im.shape[1]:
        max_updated = True
        max_shape1 = max(im.shape[1], max_shape1)
    if i == 0:
        res = im
    elif max_equal:
        res = np.concatenate(res, im)
    elif max_updated:
        pad_value = max_shape1 - res.shape[1]
        if pad_value%2 == 0:
            pads = [[0,0], [int(pad_value/2), int(pad_value/2)], [0,0]]
        else:
            pads = [[0,0], [int(pad_value/2), int(pad_value/2) + 1], [0,0]]
        res = np.concatenate([np.pad(res, pads), im])
    else:
        pad_value = max_shape1 - im.shape[1]
        if pad_value%2 == 0:
            pads = [[0,0], [int(pad_value/2), int(pad_value/2)], [0,0]]
        else:
            pads = [[0,0], [int(pad_value/2), int(pad_value/2) + 1], [0,0]]
        res = np.concatenate([res, np.pad(im, pads)])
res.shape
end = time.time()
print(end-start)

In [11]:
import time
start = time.time()
shape1s = [i.shape for i in img]
max_shape = np.max(shape1s, axis=0)
#max_shape = np.array(img[-1].shape)
def single_pad(pad_value):
    if pad_value%2 == 0:
        return [int(pad_value/2), int(pad_value/2)]
    else:
        return [int(pad_value/2), int(pad_value/2) + 1]
def calc_pad(mx, shape, axes):
    pad_values = mx - shape
    pads = [[0,0], [0,0], [0,0]]
    for axis in axes:
        #print(i)
        #print(i in axes)
        pads[axis] = (single_pad(pad_values[axis]))
    # print(pads)
    return pads
pads = [calc_pad(max_shape, i, [1]) for i in shape1s]
pads_gt = [[pad[0], pad[1]] for pad in pads]
img2 = [np.pad(img[i], pad) for i, pad in enumerate(pads)]
gts2 = [np.pad(gt[i], pad) for i, pad in enumerate(pads_gt)]
res2 = np.concatenate(img2)
gt2 = np.concatenate(gts2)
end = time.time()
print(end-start)

11.823099374771118


In [12]:
gt2.shape

(65760, 740)

In [ ]:

shape1s = [i.shape for i in img]
max_shape = np.max(shape1s, axis=0)
max_shape

In [ ]:
np.concatenate([np.ones((15,10,5)), np.pad(np.ones([14, 5, 5]), calc_pad(10,5))]).shape

In [ ]:
N_CLASSES

In [ ]:
INPUT_SIZE

In [ ]:
train_gt.shape

In [ ]:
test_gt.shape

In [ ]:
pseudo_labels3.shape

In [ ]:
X.shape

In [ ]:
Y.shape